In [13]:
import os
%pwd

'd:\\mlopsproject1\\E2E-MLops'

In [21]:
os.chdir("E2E-MLOps")

In [22]:
%pwd

'd:\\mlopsproject1\\E2E-MLOps'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig: 
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [38]:
from cnnClassifier.constants.__init__ import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [39]:
print(CONFIG_FILE_PATH)

config\config.yaml


In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [42]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [47]:
class DataIngestion:
    def __init__(self, config = DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        '''
        Fetches data from the URL
        
        '''
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok = True)
            logger.info(f"Downloading data from {dataset_url}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        '''
        zip_file_path: str
        extracts the zip file into the data directory
        Function returns None.
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [51]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-05-26 17:31:23,948: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-26 17:31:23,949: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-26 17:31:23,949: INFO: common: created directory at: artifacts]
[2025-05-26 17:31:23,950: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-26 17:31:23,951: INFO: 1111530822: Downloading data from https://drive.google.com/file/d/1WM_Qn1Mj7BGoTR2E3YgRplQA3m2j_rJZ/view?usp=sharing]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1WM_Qn1Mj7BGoTR2E3YgRplQA3m2j_rJZ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1WM_Qn1Mj7BGoTR2E3YgRplQA3m2j_rJZ&confirm=t&uuid=defad8bf-86cc-4f00-bd90-c1c5e0888353
To: d:\mlopsproject1\E2E-MLOps\artifacts\data_ingestion\data.zip
100%|██████████| 28.8M/28.8M [00:01<00:00, 22.5MB/s]

[2025-05-26 17:31:29,471: INFO: 1111530822: Downloaded data from https://drive.google.com/file/d/1WM_Qn1Mj7BGoTR2E3YgRplQA3m2j_rJZ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
